***Import all Libraries***

In [1]:

import requests
import numpy as np
import pandas as pd
import lxml.html as lh
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import json


url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')






In [3]:
tr_elements = doc.xpath('//tr')


In [5]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()

    col.append((name,[]))

In [6]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
# display the first 5 rows of the data frame
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [9]:
# drop all rows with not assigned in the borough column
df = df.set_index("Borough")
df = df.drop("Not assigned", axis=0) 



In [10]:
#reset index
df.reset_index ( inplace = True)

In [11]:
df.head ()

,Borough,Neighbourhood,Postcode
0,North York,Parkwoods\n,M3A
1,North York,Victoria Village\n,M4A
2,Downtown Toronto,Harbourfront\n,M5A
3,Downtown Toronto,Regent Park\n,M5A
4,North York,Lawrence Heights\n,M6A


In [12]:
#remove /n from dataframe
df = df.replace('\n','', regex=True)

In [13]:
df.head ()

,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


In [14]:
#rename column neighbourhood\n to Neighbourhood
df.rename(columns={'Neighbourhood\n':'Neighbourhood'}, inplace=True)

In [16]:
#group by columns
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [17]:
df.head ()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
df.shape

(103, 3)

***I have decided to use the csv file provided online to get the coordinates***

In [19]:
#read geospatial data and save it to a data frame df2
data_url ='http://cocl.us/Geospatial_data'
df2= pd.read_csv(data_url)

In [20]:
#view the data frame
df2.head ()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
#rename column postal code to postcode to match column in previous data frame
df2.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [22]:
#view the data frame to confirm the column has been renamed
df2.head ()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
#merge the two data frames on the postcode column,using the inner keyword
df3 = pd.merge(df, df2, how='inner', on = 'Postcode')

In [24]:
#view data frame to confirm if the latitude and longitude has been added to the new data frame
df3.head (3)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711


In [25]:
#new data frame to get boroughs containing toronto
df4 = df3[df3['Borough'].str.contains('Toronto', regex=False)]


In [26]:
#confirm if the two data frames have been merged
df4.head ()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [27]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df4['Borough'].unique()),
        df4.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


***I imported the folium library for visualizations***

In [30]:
!conda install -c conda-forge folium=0.5.0
import folium

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [31]:
#importing nominatim library / get the geographical coordinated of toronto
from geopy.geocoders import Nominatim
address = 'Toronto, Ca'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


***generate map for Toronto***

In [32]:
# Generating the map for Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for latitude, longitude, borough, Neighbourhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

***Authenticating on the foursquare api***

In [33]:
##get details for four square
CLIENT_ID = 'RFZ4T1WZ1J5FGETOZM1GWMGFU0C0HFG1ZKT2XMCEZOOJ4EB3' 
CLIENT_SECRET = '1SNGEGLVAS1UDRDX1RJRXS2NUCNUXPJ4OEAISJDSMNXUP0U5' 
VERSION = '20180605' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RFZ4T1WZ1J5FGETOZM1GWMGFU0C0HFG1ZKT2XMCEZOOJ4EB3
CLIENT_SECRET:1SNGEGLVAS1UDRDX1RJRXS2NUCNUXPJ4OEAISJDSMNXUP0U5


In [35]:
# getting the first neighbourhood in the data frame using the index 
df4.loc[37, 'Neighbourhood']

'The Beaches'

In [36]:
#checking the longitude and latitude of the beaches with index 37
neighborhood_latitude = df4.loc[37, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df4.loc[37, 'Longitude'] # neighborhood longitude value

neighborhood_name = df4.loc[37, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [37]:
LIMIT = 20 
# limit of number of venues returned by Foursquare API
radius = 500 
# define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RFZ4T1WZ1J5FGETOZM1GWMGFU0C0HFG1ZKT2XMCEZOOJ4EB3&client_secret=1SNGEGLVAS1UDRDX1RJRXS2NUCNUXPJ4OEAISJDSMNXUP0U5&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=20'

***Getting the results on a json file***

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca891831ed21956658dc111'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad4c062f964a52011f820e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '50aa9e744b90af0d42d5de0e',
         'name': 'Health Food Store',
         'pluralName': 'Health Food Stores',
         'primary': True,
         'shortName': 'Health Food Store'}],
       'id': '4ad4c062f964a52011f820e3',
       'location': {'address': '125 Southwood Dr',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 471,
        'formattedAddress': ['125 Southwood Dr',
         'Toronto ON M4E 0B8',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
      

In [39]:
# function to get the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**clean json file and put into a dataframe**

In [40]:

from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


**Explore various neighbourhoods in Toronto**

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Run the above function and create a new dataframe**

In [43]:
#get all the venues in toronto nearby
toronto_Venues = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [44]:
#check the data frame size
print(toronto_Venues.shape)
toronto_Venues.head()

(626, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


**Group each neighbourhood uniquely**

In [45]:
toronto_Venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13
"Cabbagetown, St. James Town",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
"Chinatown, Grange Park, Kensington Market",20,20,20,20,20,20
Christie,16,16,16,16,16,16


**Get unique categories**

In [66]:
print('There are {} uniques categories.'.format(len(toronto_Venues['Venue Category'].unique())))

There are 167 uniques categories.


**Analyze each neighbour**

In [46]:
# analysing the neighbourhoods
toronto_onehot = pd.get_dummies(toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_Venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [47]:
#check the data size
toronto_onehot.shape

(626, 168)

**group neighbours by taking the mean of the frequency of each category**

In [49]:
toronto_groups = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_groups

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.00,0.00,0.00,0.00,0.050000,0.0000,0.000000,0.00,0.0000,0.000000,0.00,0.00,0.050000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0500,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.050000,0.00,0.00,0.000000,0.05,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.05,0.0000,0.000000,0.05,0.000000,0.00,0.00,0.00,0.00,0.050000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.05,0.00,0.0000,0.05,0.05,0.00,0.000000,0.00,0.000000,0.00,0.050000,0.000000,0.00,0.05,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.05,0.000000,0.00,0.000000,0.00,0.000000,0.05,0.000000,0.000000,0.00,0.00,0.10,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.0000,0.000000,0.00,0.0000,0.000000,0.00,0.00,0.000000,0.05,0.05,0.05,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.05,0.000000,0.00,0.00,0.000000,0.05,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.100000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.05,0.000000,0.00,0.00,

**confirm the size**

In [50]:
toronto_groups.shape

(38, 168)

***PRINT EACH NEIGHBOUR WITH THE TOP 5 VENUES***

In [60]:
top_venues = 5

for hood in toronto_groups['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_groups[toronto_groups['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0        Steakhouse  0.10
1       Pizza Place  0.05
2  Greek Restaurant  0.05
3         Speakeasy  0.05
4              Café  0.05


----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1      Farmers Market  0.10
2              Museum  0.05
3            Beer Bar  0.05
4        Liquor Store  0.05


----Brockton, Exhibition Place, Parkdale Village----
                  venue  freq
0           Coffee Shop  0.10
1        Breakfast Spot  0.10
2                  Café  0.10
3  Caribbean Restaurant  0.05
4               Stadium  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Pizza Place  0.06
2          Restaurant  0.06
3                 Spa  0.06
4          Smoke Shop  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lo

In [51]:
#Sort in descending order
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

**creat a dataframe and display the top 10 venues for each neighbour**

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_groups['Neighbourhood']

for ind in np.arange(toronto_groups.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_groups.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Pizza Place,Hotel,Noodle House,Neighborhood,Concert Hall,Plaza,Café,Seafood Restaurant,Coffee Shop
1,Berczy Park,Seafood Restaurant,Farmers Market,Basketball Stadium,Steakhouse,Park,Concert Hall,Museum,Pub,Liquor Store,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Caribbean Restaurant,Falafel Restaurant,Burrito Place,Bar,Restaurant,Stadium,Grocery Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Recording Studio,Skate Park,Smoke Shop,Park,Spa,Farmers Market,Fast Food Restaurant,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Boat or Ferry,Plane,Airport


**Run K means cluster to put them into 5 neighbours**

In [52]:
kclusters = 5

toronto_grouped_clustering = toronto_groups.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 1, 3, 3, 1, 1, 1, 1, 3], dtype=int32)

**Create a new dataframe that includes the cluster with the top 10 venues for each neighbour**

In [55]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Health Food Store,Pub,Neighborhood,Dance Studio,Falafel Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Juice Bar,Dessert Shop,Bookstore,Cosmetics Shop,Pub,Fruit & Vegetable Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Sushi Restaurant,Pub,Brewery,Light Rail Station,Burger Joint,Burrito Place,Coffee Shop,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Café,Bakery,Italian Restaurant,Seafood Restaurant,Comfort Food Restaurant,Sandwich Place,Fish Market,Stationery Store,Middle Eastern Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Dance Studio,Falafel Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


**Use folium maps for visualization**

In [56]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**EXAMINE THE CLUSTERS**

***CLUSTER 1***

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,0,Bus Line,Park,Swim School,Dance Studio,Falafel Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
50,Downtown Toronto,0,Park,Playground,Trail,Cuban Restaurant,Falafel Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
64,Central Toronto,0,Trail,Park,Jewelry Store,Sushi Restaurant,Yoga Studio,Deli / Bodega,Falafel Restaurant,Eastern European Restaurant,Dog Run,Discount Store


***CLUSTER 2***

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,1,Coffee Shop,Health Food Store,Pub,Neighborhood,Dance Studio,Falafel Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner
43,East Toronto,1,Coffee Shop,Café,Bakery,Italian Restaurant,Seafood Restaurant,Comfort Food Restaurant,Sandwich Place,Fish Market,Stationery Store,Middle Eastern Restaurant
49,Central Toronto,1,Coffee Shop,Pub,Convenience Store,Sushi Restaurant,Sports Bar,Bagel Shop,Fried Chicken Joint,American Restaurant,Medical Center,Pizza Place
51,Downtown Toronto,1,Restaurant,Café,Gift Shop,Italian Restaurant,Pub,Caribbean Restaurant,Butcher,Deli / Bodega,Diner,Bakery
53,Downtown Toronto,1,Coffee Shop,Park,Breakfast Spot,Bakery,Chocolate Shop,Pub,Restaurant,Dessert Shop,Mexican Restaurant,Performing Arts Venue
55,Downtown Toronto,1,Gastropub,Coffee Shop,Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Performing Arts Venue,Café,Japanese Restaurant,BBQ Joint,Italian Restaurant
57,Downtown Toronto,1,Coffee Shop,Pizza Place,Sushi Restaurant,Chinese Restaurant,Modern European Restaurant,Ramen Restaurant,Bubble Tea Shop,Sandwich Place,Seafood Restaurant,Bar
60,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Gym,Hotel,Japanese Restaurant,Deli / Bodega,Pub,Beer Bar,Art Gallery
61,Downtown Toronto,1,Café,Gastropub,Coffee Shop,Tea Room,Museum,Pub,Deli / Bodega,Hotel,Restaurant,Art Gallery
67,Downtown Toronto,1,Café,Vietnamese Restaurant,Beer Bar,Italian Restaurant,Coffee Shop,Cocktail Bar,Cheese Shop,Caribbean Restaurant,Dessert Shop,Mexican Restaurant


***CLUSTER 3***

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Garden,Yoga Studio,Dance Studio,Farmers Market,Falafel Restaurant,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


***CLUSTER 4***

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,3,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Juice Bar,Dessert Shop,Bookstore,Cosmetics Shop,Pub,Fruit & Vegetable Store
42,East Toronto,3,Sushi Restaurant,Pub,Brewery,Light Rail Station,Burger Joint,Burrito Place,Coffee Shop,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant
45,Central Toronto,3,Gym,Hotel,Burger Joint,Sandwich Place,Food & Drink Shop,Dance Studio,Asian Restaurant,Breakfast Spot,Pizza Place,Park
46,Central Toronto,3,Sporting Goods Shop,Coffee Shop,Yoga Studio,Fast Food Restaurant,Diner,Dessert Shop,Metro Station,Mexican Restaurant,Park,Clothing Store
47,Central Toronto,3,Dessert Shop,Sushi Restaurant,Italian Restaurant,Pizza Place,Coffee Shop,Café,Restaurant,Brewery,Diner,Sandwich Place
52,Downtown Toronto,3,Pizza Place,Park,Bubble Tea Shop,Salon / Barbershop,Burger Joint,Restaurant,Ramen Restaurant,Pub,Coffee Shop,Bookstore
54,Downtown Toronto,3,Café,Pizza Place,Taco Place,Music Venue,Comic Shop,Movie Theater,Plaza,Ramen Restaurant,Burrito Place,Burger Joint
56,Downtown Toronto,3,Seafood Restaurant,Farmers Market,Basketball Stadium,Steakhouse,Park,Concert Hall,Museum,Pub,Liquor Store,Breakfast Spot
58,Downtown Toronto,3,Steakhouse,Pizza Place,Hotel,Noodle House,Neighborhood,Concert Hall,Plaza,Café,Seafood Restaurant,Coffee Shop
59,Downtown Toronto,3,Park,Café,Hotel,Plaza,Bubble Tea Shop,Sports Bar,Supermarket,Sporting Goods Shop,Lake,Deli / Bodega


***CLUSTER 5***

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,4,Playground,Restaurant,Gym,Tennis Court,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega


***End***